In [1]:
import copy

import numpy as np
import pandas as pd
from itertools import combinations

from sklearn.model_selection import KFold
from skopt.space import Real

import sys
sys.path.append('G:/내 드라이브/ensemble kernel')

from Kernel_Function_3 import split_data, prepare_response_variable, c_index_kernel_type

## Preprocessing

In [ ]:
data = pd.read_csv("G:/내 드라이브/ensemble kernel/icu_pneu.csv")
data

In [64]:
data = pd.read_csv("G:/내 드라이브/ensemble kernel/icu_pneu.csv")

# time(OS) 열 생성
data['OS'] = data['stop'] - data['start']

df = data.drop(columns=['Unnamed: 0','id', 'start', 'stop']).dropna()
df.rename(columns = {'status':'Status', 'time':'OS', 'age':'Age', 'sex':'Sex'}, inplace=True)
df.replace({'Sex' : {'F':0, 'M':1}}, inplace=True)

columns_to_convert = ['event','pneu','Sex']
df[columns_to_convert] = df[columns_to_convert].astype('category')

#onehot encode
df_onehot = pd.get_dummies(df, drop_first=True).rename(columns={'Sex_1':'Sex', 'pneu_1':'pneu'})

columns_to_convert = ['event_3','pneu','Sex']
df_onehot[columns_to_convert] = df_onehot[columns_to_convert].astype('category')
df_onehot

,Status,adm.cens.exit,Age,OS,event_3,pneu,Sex
0,1,421.0,62.533071,3.0,1,0,0
1,1,545.0,75.341530,5.0,1,0,0
2,1,372.0,76.103825,88.0,1,0,1
3,1,373.0,39.789617,8.0,1,0,0
4,1,372.0,19.173800,25.0,1,0,1
...,...,...,...,...,...,...,...
1416,1,413.0,61.442600,21.0,1,0,0
1417,1,445.0,69.851059,21.0,1,0,1
1418,1,537.0,60.232570,49.0,1,0,1
1419,1,555.0,59.521858,35.0,1,0,1


### K-fold setting

In [65]:
cv = KFold(n_splits=5, shuffle=True, random_state=36)

param_grid = {'alpha': 2. ** np.arange(-12, 13, 2)}
param_space = {'alpha': Real(1e-6, 1e+6, 'log-uniform'),}

### Selection of 100 random state numbers

In [67]:
import random

with open("G:/내 드라이브/ensemble kernel/random_state_100.txt", "r") as file:
    random_state=file.read()
    
random_state=random_state.split("\n")
random_state=[int(x) for x in random_state if x]

## Scenario 1

#### Linear

In [138]:
linear_results=pd.DataFrame()

linear_results['train_C_index']=[]
linear_results['test_C_index']=[]

In [139]:
kernel_type = 'linear'

for i in range(100):
    
    x_train, x_test, target_train, target_test = split_data(df_onehot, randomState = random_state[i])

    y_train = prepare_response_variable(target_train)
    y_test = prepare_response_variable(target_test)

    results = c_index_kernel_type(x_train, y_train, x_test, y_test, param_grid, param_space, cv, keywords = ['Age', 'Sex'], type = kernel_type)
    
    linear_results = linear_results.append({"train_C_index":results[0],"test_C_index":results[1]}, ignore_index=True)

linear_results.to_csv("G:/내 드라이브/ensemble kernel/icu.pneu_linear.csv", index = False, encoding = 'cp949')


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_39564\1035045978.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  linear_results = linear_results.append({"train_C_index":results[0],"test_C_index":results[1]}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_39564\1035045978.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  linear_results = linear_results.append({"train_C_index":results[0],"test_C_index":results[1]}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_39564\1035045978.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  linear_results = linear_results.append({"train_C_index":results[0],"test_C_index":results[1]}, ignore_index=True)
C:\Users\Publ

#### Clinical

In [140]:
clinical_results=pd.DataFrame()

clinical_results['train_C_index']=[]
clinical_results['test_C_index']=[]

In [141]:
kernel_type = 'clinical'

for i in range(100):
    
    x_train, x_test, target_train, target_test = split_data(df_onehot, randomState = random_state[i])

    y_train = prepare_response_variable(target_train)
    y_test = prepare_response_variable(target_test)

    results = c_index_kernel_type(x_train, y_train, x_test, y_test, param_grid, param_space, cv, keywords = ['Age', 'Sex'], type = kernel_type)
    
    clinical_results = clinical_results.append({"train_C_index":results[0],"test_C_index":results[1]}, ignore_index=True)

clinical_results.to_csv("G:/내 드라이브/ensemble kernel/icu.pneu_clinical.csv", index = False, encoding = 'cp949')

c:\Users\stat\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_search.py:910: ConvergenceWarning: Optimization did not converge: Warning: CG iterations didn't converge. The Hessian is not positive definite.
  self.best_estimator_.fit(X, y, **fit_params)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_39564\3913638445.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  clinical_results = clinical_results.append({"train_C_index":results[0],"test_C_index":results[1]}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_39564\3913638445.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  clinical_results = clinical_results.append({"train_C_index":results[0],"test_C_index":results[1]}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\i

#### Ensemble Cox

In [142]:
ensemble_cox_results=pd.DataFrame()

ensemble_cox_results['train_C_index']=[]
ensemble_cox_results['test_C_index']=[]

In [143]:
kernel_type = 'ensemble_cox'

for i in range(100):
    
    x_train, x_test, target_train, target_test = split_data(df_onehot, randomState = random_state[i])

    y_train = prepare_response_variable(target_train)
    y_test = prepare_response_variable(target_test)

    results = c_index_kernel_type(x_train, y_train, x_test, y_test, param_grid, param_space, cv, keywords = ['Age', 'Sex'], type = kernel_type)
    
    ensemble_cox_results = ensemble_cox_results.append({"train_C_index":results[0],"test_C_index":results[1]}, ignore_index=True)

ensemble_cox_results.to_csv("G:/내 드라이브/ensemble kernel/icu.pneu_cox.csv", index = False, encoding = 'cp949')

c:\Users\stat\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_search.py:910: ConvergenceWarning: Optimization did not converge: Warning: Desired error not necessarily achieved due to precision loss.
  self.best_estimator_.fit(X, y, **fit_params)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_39564\1831736452.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ensemble_cox_results = ensemble_cox_results.append({"train_C_index":results[0],"test_C_index":results[1]}, ignore_index=True)
c:\Users\stat\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_search.py:910: ConvergenceWarning: Optimization did not converge: Warning: Desired error not necessarily achieved due to precision loss.
  self.best_estimator_.fit(X, y, **fit_params)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_39564\1831736452.py:12: FutureWarning: The 

#### Ensemble AFT

In [144]:
ensemble_aft_results=pd.DataFrame()

ensemble_aft_results['train_C_index']=[]
ensemble_aft_results['test_C_index']=[]

In [145]:
kernel_type = 'ensemble_aft'

for i in range(100):
    
    x_train, x_test, target_train, target_test = split_data(df_onehot, randomState = random_state[i])

    y_train = prepare_response_variable(target_train)
    y_test = prepare_response_variable(target_test)

    results = c_index_kernel_type(x_train, y_train, x_test, y_test, param_grid, param_space, cv, keywords = ['Age', 'Sex'], type = kernel_type)
    
    ensemble_aft_results = ensemble_aft_results.append({"train_C_index":results[0],"test_C_index":results[1]}, ignore_index=True)

ensemble_aft_results.to_csv("G:/내 드라이브/ensemble kernel/icu.pneu_aft.csv", index = False, encoding = 'cp949')

c:\Users\stat\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_search.py:910: ConvergenceWarning: Optimization did not converge: Warning: Desired error not necessarily achieved due to precision loss.
  self.best_estimator_.fit(X, y, **fit_params)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_39564\2200913932.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ensemble_aft_results = ensemble_aft_results.append({"train_C_index":results[0],"test_C_index":results[1]}, ignore_index=True)
c:\Users\stat\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_search.py:910: ConvergenceWarning: Optimization did not converge: Warning: CG iterations didn't converge. The Hessian is not positive definite.
  self.best_estimator_.fit(X, y, **fit_params)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_39564\2200913932.py:12: FutureWarnin

### Result

In [146]:
linear_results = pd.read_csv('G:/내 드라이브/ensemble kernel/icu.pneu_linear.csv')
clinical_results = pd.read_csv('G:/내 드라이브/ensemble kernel/icu.pneu_clinical.csv')
ensemble_cox_results = pd.read_csv('G:/내 드라이브/ensemble kernel/icu.pneu_cox.csv')
ensemble_aft_results = pd.read_csv('G:/내 드라이브/ensemble kernel/icu.pneu_aft.csv')

### The value of C-index

In [147]:
print(round(np.mean(linear_results['train_C_index']),4))
print(round(np.std(linear_results['train_C_index']),4))
print(round(np.mean(linear_results['test_C_index']),4))
print(round(np.std(linear_results['test_C_index']),4))

0.5664
0.0068
0.5657
0.0137


In [148]:
print(round(np.mean(clinical_results['train_C_index']),4))
print(round(np.std(clinical_results['train_C_index']),4))
print(round(np.mean(clinical_results['test_C_index']),4))
print(round(np.std(clinical_results['test_C_index']),4))

0.6448
0.041
0.5853
0.0139


In [149]:
print(round(np.mean(ensemble_cox_results['train_C_index']),4))
print(round(np.std(ensemble_cox_results['train_C_index']),4))
print(round(np.mean(ensemble_cox_results['test_C_index']),4))
print(round(np.std(ensemble_cox_results['test_C_index']),4))

0.6168
0.0172
0.5848
0.0141


In [150]:
print(round(np.mean(ensemble_aft_results['train_C_index']),4))
print(round(np.std(ensemble_aft_results['train_C_index']),4))
print(round(np.mean(ensemble_aft_results['test_C_index']),4))
print(round(np.std(ensemble_aft_results['test_C_index']),4))

0.619
0.0175
0.5853
0.014


## Scenario 2

In [71]:
# Remaining variable results from 100 runs
cox_remaining_variable=pd.DataFrame()

cox_remaining_variable['variable']=[]

### CV of selection of variables

In [72]:
for i in range(100):
    x_train, x_test, target_train, target_test = split_data(df_onehot, randomState = random_state[i])

    y_train = prepare_response_variable(target_train)
    y_test = prepare_response_variable(target_test)

    # Separating OS and Status
    drop = x_train[['Sex', 'Age', 'OS', 'Status']]
    x_train_drop = x_train.drop(columns=['Sex', 'Age', 'OS','Status'])

    # Save column names of a DataFrame into a list
    columns = x_train_drop.columns

    column_groups = []

    for i in range(len(columns)):
        # Generate all combinations of columns taken i+1 at a time
        all_column_combinations = list(combinations(columns, i+1))

        # Create a DataFrame for each combination
        for column_combination in all_column_combinations:
            selected_columns = list(column_combination)
            sub_train1 = x_train_drop[selected_columns]
            sub_train2 = pd.concat([sub_train1, drop], axis=1)
            column_groups.append(sub_train2)
    
    num_groups = 3
    best_cindex = []

    for i in range(len(column_groups)):
        x_groups = []
        train = column_groups[i].sample(frac=1).reset_index(drop=True)
        
        censored = train[train['Status'] == 0]  # Extract rows with a value of 0
        uncensored = train[train['Status'] == 1]  # Extract rows with a value of 1

        group_size1 = len(censored) // num_groups
        group_size2 = len(uncensored) // num_groups

        # Split train data into groups
        for i in range(num_groups):
                
            if i < num_groups - 1:
                # Adjusting the censoring ratio
                group1 = censored.iloc[i * group_size1:(i + 1) * group_size1]
                group2 = uncensored.iloc[i * group_size2:(i + 1) * group_size2]

                group = pd.concat([group1, group2], ignore_index=True)
            else:
                group1 = censored.iloc[i * group_size1:]
                group2 = uncensored.iloc[i * group_size2:]

                group = pd.concat([group1, group2], ignore_index=True)
    
            x_groups.append(group)

        cindex = []

        for i in range(len(x_groups)):

            temp = []
            for j in range(len(x_groups)):
                if i != j:
                    temp.append(x_groups[j])
            # train = train data(One out of the equally divided segments, excluding one)
            train = pd.concat(temp)

            # validation = validation data(One of the equally divided segments)
            validation = x_groups[i]
            
            train_target = train[['Status','OS']]
            test_target = validation[['Status','OS']]

            train_y = prepare_response_variable(train_target)
            test_y = prepare_response_variable(test_target)
            #Define data -> train, train_y, validation,  test_y

            result = c_index_kernel_type(train, train_y, validation, test_y, param_grid, param_space, cv, keywords = ['Age', 'Sex'], type = 'ensemble_cox')
            cindex.append(result[1])

        #best_cindex: Store all validation c-index values for each selected variable as a list
        best_cindex.append(np.mean(cindex))

    #max_num: Index number of the best c-index in the list
    max_num = best_cindex.index(max(best_cindex))

    #train_column: Selected variables for the best c-index
    train_column = column_groups[max_num].columns
    
    cox_remaining_variable = cox_remaining_variable.append({"variable": list(train_column)}, ignore_index = True)



c:\Users\stat\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_search.py:910: ConvergenceWarning: Optimization did not converge: Warning: Desired error not necessarily achieved due to precision loss.
  self.best_estimator_.fit(X, y, **fit_params)
c:\Users\stat\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_search.py:910: ConvergenceWarning: Optimization did not converge: Warning: Desired error not necessarily achieved due to precision loss.
  self.best_estimator_.fit(X, y, **fit_params)
c:\Users\stat\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_search.py:910: ConvergenceWarning: Optimization did not converge: Warning: Desired error not necessarily achieved due to precision loss.
  self.best_estimator_.fit(X, y, **fit_params)
c:\Users\stat\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_search.py:910: ConvergenceWarning: Optimization did not converge

In [87]:
cox_remaining_variable.to_csv("G:/내 드라이브/ensemble kernel/icu.pneu_cox_remaining_variables.csv", index = False, encoding = 'cp949')

### C-index by Kernel type

##### Linear Kernel

In [79]:
linear_results=pd.DataFrame()

linear_results['train_C_index']=[]
linear_results['test_C_index']=[]

In [82]:
kernel_type = 'linear'

for i in range(100):
    variables = eval(pd.read_csv("G:/내 드라이브/ensemble kernel/icu.pneu_cox_remaining_variables.csv")['variable'][i])

    # variables.append('Status')
    # variables.append('OS')

    df_onehot_re = df_onehot[variables]
    
    x_train, x_test, target_train, target_test = split_data(df_onehot_re, randomState = random_state[i])

    y_train = prepare_response_variable(target_train)
    y_test = prepare_response_variable(target_test)

    results = c_index_kernel_type(x_train, y_train, x_test, y_test, param_grid, param_space, cv, keywords = ['Age', 'Sex'], type = kernel_type)
    
    linear_results = linear_results.append({"train_C_index":results[0],"test_C_index":results[1]}, ignore_index=True)

linear_results.to_csv("G:/내 드라이브/ensemble kernel/icu.pneu_cox_linear.csv", index = False, encoding = 'cp949')


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_39564\3542812305.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  linear_results = linear_results.append({"train_C_index":results[0],"test_C_index":results[1]}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_39564\3542812305.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  linear_results = linear_results.append({"train_C_index":results[0],"test_C_index":results[1]}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_39564\3542812305.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  linear_results = linear_results.append({"train_C_index":results[0],"test_C_index":results[1]}, ignore_index=True)
C:\Users\Publ

##### Clinical Kernel

In [90]:
clinical_results=pd.DataFrame()

clinical_results['train_C_index']=[]
clinical_results['test_C_index']=[]

In [91]:
kernel_type = 'clinical'

for i in range(100):
    #variables = all_results['remaining_variables'][i]
    variables = eval(pd.read_csv("G:/내 드라이브/ensemble kernel/icu.pneu_cox_remaining_variables.csv")['variable'][i])

    # variables.append('Status')
    # variables.append('OS')

    df_onehot_re = df_onehot[variables]
    
    x_train, x_test, target_train, target_test = split_data(df_onehot_re, randomState = random_state[i])

    y_train = prepare_response_variable(target_train)
    y_test = prepare_response_variable(target_test)

    results = c_index_kernel_type(x_train, y_train, x_test, y_test, param_grid, param_space, cv, keywords = ['Age', 'Sex'], type = kernel_type)
    
    clinical_results = clinical_results.append({"train_C_index":results[0],"test_C_index":results[1]}, ignore_index=True)

clinical_results.to_csv("G:/내 드라이브/ensemble kernel/icu.pneu_cox_clinical.csv", index = False, encoding = 'cp949')

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_39564\436065203.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  clinical_results = clinical_results.append({"train_C_index":results[0],"test_C_index":results[1]}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_39564\436065203.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  clinical_results = clinical_results.append({"train_C_index":results[0],"test_C_index":results[1]}, ignore_index=True)
c:\Users\stat\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_search.py:910: ConvergenceWarning: Optimization did not converge: Warning: Desired error not necessarily achieved due to precision loss.
  self.best_estimator_.fit(X, y, **fit_params)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_

##### Ensemble Cox Kernel

In [92]:
ensemble_cox_results=pd.DataFrame()

ensemble_cox_results['train_C_index']=[]
ensemble_cox_results['test_C_index']=[]

In [94]:
kernel_type = 'ensemble_cox'

for i in range(100):
    #variables = all_results['remaining_variables'][i]
    variables = eval(pd.read_csv("G:/내 드라이브/ensemble kernel/icu.pneu_cox_remaining_variables.csv")['variable'][i])

    # variables.append('Status')
    # variables.append('OS')

    df_onehot_re = df_onehot[variables]
    
    x_train, x_test, target_train, target_test = split_data(df_onehot_re, randomState = random_state[i])

    y_train = prepare_response_variable(target_train)
    y_test = prepare_response_variable(target_test)

    results = c_index_kernel_type(x_train, y_train, x_test, y_test, param_grid, param_space, cv, keywords = ['Age', 'Sex'], type = kernel_type)
    
    ensemble_cox_results = ensemble_cox_results.append({"train_C_index":results[0],"test_C_index":results[1]}, ignore_index=True)

ensemble_cox_results.to_csv("G:/내 드라이브/ensemble kernel/icu.pneu_cox_cox.csv", index = False, encoding = 'cp949')

c:\Users\stat\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_search.py:910: ConvergenceWarning: Optimization did not converge: Warning: Desired error not necessarily achieved due to precision loss.
  self.best_estimator_.fit(X, y, **fit_params)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_39564\1407574221.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ensemble_cox_results = ensemble_cox_results.append({"train_C_index":results[0],"test_C_index":results[1]}, ignore_index=True)
c:\Users\stat\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_search.py:910: ConvergenceWarning: Optimization did not converge: Warning: Desired error not necessarily achieved due to precision loss.
  self.best_estimator_.fit(X, y, **fit_params)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_39564\1407574221.py:19: FutureWarning: The 

##### Ensemble AFT Kernel

In [95]:
ensemble_aft_results=pd.DataFrame()

ensemble_aft_results['train_C_index']=[]
ensemble_aft_results['test_C_index']=[]

In [96]:
kernel_type = 'ensemble_aft'

for i in range(100):
    #variables = all_results['remaining_variables'][i]
    variables = eval(pd.read_csv("G:/내 드라이브/ensemble kernel/icu.pneu_cox_remaining_variables.csv")['variable'][i])

    # variables.append('Status')
    # variables.append('OS')

    df_onehot_re = df_onehot[variables]
    
    x_train, x_test, target_train, target_test = split_data(df_onehot_re, randomState = random_state[i])

    y_train = prepare_response_variable(target_train)
    y_test = prepare_response_variable(target_test)

    results = c_index_kernel_type(x_train, y_train, x_test, y_test, param_grid, param_space, cv, keywords = ['Age', 'Sex'], type = kernel_type)
    
    ensemble_aft_results = ensemble_aft_results.append({"train_C_index":results[0],"test_C_index":results[1]}, ignore_index=True)

ensemble_aft_results.to_csv("G:/내 드라이브/ensemble kernel/icu.pneu_cox_aft.csv", index = False, encoding = 'cp949')

c:\Users\stat\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_search.py:910: ConvergenceWarning: Optimization did not converge: Warning: Desired error not necessarily achieved due to precision loss.
  self.best_estimator_.fit(X, y, **fit_params)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_39564\3880916993.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ensemble_aft_results = ensemble_aft_results.append({"train_C_index":results[0],"test_C_index":results[1]}, ignore_index=True)
c:\Users\stat\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_search.py:910: ConvergenceWarning: Optimization did not converge: Warning: Desired error not necessarily achieved due to precision loss.
  self.best_estimator_.fit(X, y, **fit_params)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_39564\3880916993.py:19: FutureWarning: The 

#### Result

In [97]:
linear_results = pd.read_csv('G:/내 드라이브/ensemble kernel/icu.pneu_cox_linear.csv')
clinical_results = pd.read_csv('G:/내 드라이브/ensemble kernel/icu.pneu_cox_clinical.csv')
ensemble_cox_results = pd.read_csv('G:/내 드라이브/ensemble kernel/icu.pneu_cox_cox.csv')
ensemble_aft_results = pd.read_csv('G:/내 드라이브/ensemble kernel/icu.pneu_cox_aft.csv')

#### Count of selections for remaining variables

In [99]:
remaining_variables_all=[]
for i in range(0,100):
    remaining_variables_all+=eval(pd.read_csv("G:/내 드라이브/ensemble kernel/icu.pneu_cox_remaining_variables.csv")['variable'][i])

element_counts = {}

for element in remaining_variables_all:
    if element in element_counts:
        element_counts[element] += 1
    else:
        element_counts[element] = 1

print(element_counts)

{'adm.cens.exit': 100, 'pneu': 97, 'Sex': 100, 'Age': 100, 'OS': 100, 'Status': 100, 'event_3': 38}


### The value of C-index

In [100]:
print(round(np.mean(linear_results['train_C_index']),4))
print(round(np.std(linear_results['train_C_index']),4))
print(round(np.mean(linear_results['test_C_index']),4))
print(round(np.std(linear_results['test_C_index']),4))

0.5664
0.0068
0.5657
0.0137


In [101]:
print(round(np.mean(clinical_results['train_C_index']),4))
print(round(np.std(clinical_results['train_C_index']),4))
print(round(np.mean(clinical_results['test_C_index']),4))
print(round(np.std(clinical_results['test_C_index']),4))

0.6409
0.0425
0.5844
0.0138


In [102]:
print(round(np.mean(ensemble_cox_results['train_C_index']),4))
print(round(np.std(ensemble_cox_results['train_C_index']),4))
print(round(np.mean(ensemble_cox_results['test_C_index']),4))
print(round(np.std(ensemble_cox_results['test_C_index']),4))

0.6121
0.0172
0.5844
0.0141


In [103]:
print(round(np.mean(ensemble_aft_results['train_C_index']),4))
print(round(np.std(ensemble_aft_results['train_C_index']),4))
print(round(np.mean(ensemble_aft_results['test_C_index']),4))
print(round(np.std(ensemble_aft_results['test_C_index']),4))

0.6123
0.0174
0.5842
0.0139


## Scenario 3

In [104]:
# Remaining variable results from 100 runs
aft_remaining_variable=pd.DataFrame()

aft_remaining_variable['variable']=[]

In [110]:
df_onehot.columns

Index(['Status', 'adm.cens.exit', 'Age', 'OS', 'event_3', 'pneu', 'Sex'], dtype='object')

### CV of selection of variables

In [118]:
for i in range(100):
    x_train, x_test, target_train, target_test = split_data(df_onehot, randomState = random_state[i])

    y_train = prepare_response_variable(target_train)
    y_test = prepare_response_variable(target_test)

    # Separating OS and Status
    drop = x_train[['Sex', 'Age', 'OS','Status']]
    x_train_drop = x_train.drop(columns=['Sex', 'Age', 'OS','Status'])

    # Save column names of a DataFrame into a list
    columns = x_train_drop.columns

    column_groups = []

    for i in range(len(columns)):
        # Generate all combinations of columns taken i+1 at a time
        all_column_combinations = list(combinations(columns, i+1))

        # Create a DataFrame for each combination
        for column_combination in all_column_combinations:
            selected_columns = list(column_combination)
            sub_train1 = x_train_drop[selected_columns]
            sub_train2 = pd.concat([sub_train1, drop], axis=1)
            column_groups.append(sub_train2)
    
    num_groups = 3
    best_cindex = []

    for i in range(len(column_groups)):
        x_groups = []
        train = column_groups[i].sample(frac=1).reset_index(drop=True)
        
        censored = train[train['Status'] == 0]  # Extract rows with a value of 0
        uncensored = train[train['Status'] == 1]  # Extract rows with a value of 1

        group_size1 = len(censored) // num_groups
        group_size2 = len(uncensored) // num_groups

        # Splitting the train data into groups
        for i in range(num_groups):

            if i < num_groups - 1:
                # Adjusting the censoring ratio
                group1 = censored.iloc[i * group_size1:(i + 1) * group_size1]
                group2 = uncensored.iloc[i * group_size2:(i + 1) * group_size2]

                group = pd.concat([group1, group2], ignore_index=True)
            else:
                group1 = censored.iloc[i * group_size1:]
                group2 = uncensored.iloc[i * group_size2:]

                group = pd.concat([group1, group2], ignore_index=True)
    
            x_groups.append(group)

        cindex = []

        for i in range(len(x_groups)):

            temp = []
            for j in range(len(x_groups)):
                if i != j:
                    temp.append(x_groups[j])
            #train = train data(One out of the equally divided segments, excluding one)
            train=pd.concat(temp, ignore_index=True)

            #validation = validation data(One of the equally divided segments)
            validation = x_groups[i]
            
            train_target=train[['Status','OS']]
            test_target=validation[['Status','OS']]

            train_y=prepare_response_variable(train_target)
            test_y=prepare_response_variable(test_target)
            #Define data -> train, train_y, validation,  test_y

            result = c_index_kernel_type(train, train_y, validation, test_y, param_grid, param_space, cv, keywords = ['Age', 'Sex'], type = 'ensemble_aft')
            cindex.append(result[1])

        #best_cindex: Store all validation c-index values for each selected variable as a list
        best_cindex.append(np.mean(cindex))

    #max_num: Index number of the best c-index in the list
    max_num = best_cindex.index(max(best_cindex))

    #train_column: Selected variables for the best c-index
    train_column = column_groups[max_num].columns
    
    aft_remaining_variable = aft_remaining_variable.append({"variable": list(train_column)}, ignore_index = True)



c:\Users\stat\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_search.py:910: ConvergenceWarning: Optimization did not converge: Warning: Desired error not necessarily achieved due to precision loss.
  self.best_estimator_.fit(X, y, **fit_params)
c:\Users\stat\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_search.py:910: ConvergenceWarning: Optimization did not converge: Warning: Desired error not necessarily achieved due to precision loss.
  self.best_estimator_.fit(X, y, **fit_params)
c:\Users\stat\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_search.py:910: ConvergenceWarning: Optimization did not converge: Warning: Desired error not necessarily achieved due to precision loss.
  self.best_estimator_.fit(X, y, **fit_params)
c:\Users\stat\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_search.py:910: ConvergenceWarning: Optimization did not converge

In [119]:
aft_remaining_variable.to_csv("G:/내 드라이브/ensemble kernel/icu.pneu_aft_remaining_variables.csv", index = False, encoding = 'cp949')

### C-index by Kernel type

#### Linear

In [135]:
linear_results=pd.DataFrame()

linear_results['train_C_index']=[]
linear_results['test_C_index']=[]

In [136]:
kernel_type = 'linear'

for i in range(100):
    #variables = all_results['remaining_variables'][i]
    variables = eval(pd.read_csv("G:/내 드라이브/ensemble kernel/icu.pneu_aft_remaining_variables.csv")['variable'][i])

    df_onehot_re = df_onehot[variables]
    
    x_train, x_test, target_train, target_test = split_data(df_onehot_re, randomState = random_state[i])

    y_train = prepare_response_variable(target_train)
    y_test = prepare_response_variable(target_test)

    results = c_index_kernel_type(x_train, y_train, x_test, y_test, param_grid, param_space, cv, keywords = ['Age', 'Sex'], type = kernel_type)
    
    linear_results = linear_results.append({"train_C_index":results[0],"test_C_index":results[1]}, ignore_index=True)

linear_results.to_csv("G:/내 드라이브/ensemble kernel/icu.pneu_aft_linear.csv", index = False, encoding = 'cp949')

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_39564\526575449.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  linear_results = linear_results.append({"train_C_index":results[0],"test_C_index":results[1]}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_39564\526575449.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  linear_results = linear_results.append({"train_C_index":results[0],"test_C_index":results[1]}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_39564\526575449.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  linear_results = linear_results.append({"train_C_index":results[0],"test_C_index":results[1]}, ignore_index=True)
C:\Users\Public\

#### Clinical 

In [122]:
clinical_results=pd.DataFrame()

clinical_results['train_C_index']=[]
clinical_results['test_C_index']=[]

In [123]:
kernel_type = 'clinical'

for i in range(100):
    #variables = all_results['remaining_variables'][i]
    variables = eval(pd.read_csv("G:/내 드라이브/ensemble kernel/icu.pneu_aft_remaining_variables.csv")['variable'][i])

    df_onehot_re = df_onehot[variables]
    
    x_train, x_test, target_train, target_test = split_data(df_onehot_re, randomState = random_state[i])

    y_train = prepare_response_variable(target_train)
    y_test = prepare_response_variable(target_test)

    results = c_index_kernel_type(x_train, y_train, x_test, y_test, param_grid, param_space, cv, keywords = ['Age', 'Sex'], type = kernel_type)
    
    clinical_results = clinical_results.append({"train_C_index":results[0],"test_C_index":results[1]}, ignore_index=True)

clinical_results.to_csv("G:/내 드라이브/ensemble kernel/icu.pneu_aft_clinical.csv", index = False, encoding = 'cp949')

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_39564\3560900555.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  clinical_results = clinical_results.append({"train_C_index":results[0],"test_C_index":results[1]}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_39564\3560900555.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  clinical_results = clinical_results.append({"train_C_index":results[0],"test_C_index":results[1]}, ignore_index=True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_39564\3560900555.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  clinical_results = clinical_results.append({"train_C_index":results[0],"test_C_index":results[1]}, ignore_index=True)
C

#### Ensemble Cox

In [124]:
ensemble_cox_results=pd.DataFrame()

ensemble_cox_results['train_C_index']=[]
ensemble_cox_results['test_C_index']=[]

In [125]:
kernel_type = 'ensemble_cox'

for i in range(100):
    #variables = all_results['remaining_variables'][i]
    variables = eval(pd.read_csv("G:/내 드라이브/ensemble kernel/icu.pneu_aft_remaining_variables.csv")['variable'][i])

    df_onehot_re = df_onehot[variables]
    
    x_train, x_test, target_train, target_test = split_data(df_onehot_re, randomState = random_state[i])

    y_train = prepare_response_variable(target_train)
    y_test = prepare_response_variable(target_test)

    results = c_index_kernel_type(x_train, y_train, x_test, y_test, param_grid, param_space, cv, keywords = ['Age', 'Sex'], type = kernel_type)
    
    ensemble_cox_results = ensemble_cox_results.append({"train_C_index":results[0],"test_C_index":results[1]}, ignore_index=True)

ensemble_cox_results.to_csv("G:/내 드라이브/ensemble kernel/icu.pneu_aft_cox.csv", index = False, encoding = 'cp949')

c:\Users\stat\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_search.py:910: ConvergenceWarning: Optimization did not converge: Warning: Desired error not necessarily achieved due to precision loss.
  self.best_estimator_.fit(X, y, **fit_params)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_39564\823211529.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ensemble_cox_results = ensemble_cox_results.append({"train_C_index":results[0],"test_C_index":results[1]}, ignore_index=True)
c:\Users\stat\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_search.py:910: ConvergenceWarning: Optimization did not converge: Warning: Desired error not necessarily achieved due to precision loss.
  self.best_estimator_.fit(X, y, **fit_params)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_39564\823211529.py:16: FutureWarning: The fr

#### Ensemble AFT

In [126]:
ensemble_aft_results=pd.DataFrame()

ensemble_aft_results['train_C_index']=[]
ensemble_aft_results['test_C_index']=[]

In [127]:
kernel_type = 'ensemble_aft'

for i in range(100):
    #variables = remaining_variable[i]
    variables = eval(pd.read_csv("G:/내 드라이브/ensemble kernel/icu.pneu_aft_remaining_variables.csv")['variable'][i])

    df_onehot_re = df_onehot[variables]
    
    x_train, x_test, target_train, target_test = split_data(df_onehot_re, randomState = random_state[i])

    y_train = prepare_response_variable(target_train)
    y_test = prepare_response_variable(target_test)

    results = c_index_kernel_type(x_train, y_train, x_test, y_test, param_grid, param_space, cv, keywords = ['Age', 'Sex'], type = kernel_type)
    
    ensemble_aft_results = ensemble_aft_results.append({"train_C_index":results[0],"test_C_index":results[1]}, ignore_index=True)

ensemble_aft_results.to_csv("G:/내 드라이브/ensemble kernel/icu.pneu_aft_aft.csv", index = False, encoding = 'cp949')

c:\Users\stat\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_search.py:910: ConvergenceWarning: Optimization did not converge: Warning: Desired error not necessarily achieved due to precision loss.
  self.best_estimator_.fit(X, y, **fit_params)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_39564\542980023.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ensemble_aft_results = ensemble_aft_results.append({"train_C_index":results[0],"test_C_index":results[1]}, ignore_index=True)
c:\Users\stat\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_search.py:910: ConvergenceWarning: Optimization did not converge: Warning: Desired error not necessarily achieved due to precision loss.
  self.best_estimator_.fit(X, y, **fit_params)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_39564\542980023.py:16: FutureWarning: The fr

### Result

In [128]:
linear_results = pd.read_csv('G:/내 드라이브/ensemble kernel/icu.pneu_aft_linear.csv')
clinical_results = pd.read_csv('G:/내 드라이브/ensemble kernel/icu.pneu_aft_clinical.csv')
ensemble_cox_results = pd.read_csv('G:/내 드라이브/ensemble kernel/icu.pneu_aft_cox.csv')
ensemble_aft_results = pd.read_csv('G:/내 드라이브/ensemble kernel/icu.pneu_aft_aft.csv')

### Count of Selections for remaining variables

In [129]:
remaining_variables_all=[]
for i in range(0,100):
    remaining_variables_all += eval(pd.read_csv("G:/내 드라이브/ensemble kernel/icu.pneu_aft_remaining_variables.csv")['variable'][i])

element_counts = {}

for element in remaining_variables_all:
    if element in element_counts:
        element_counts[element] += 1
    else:
        element_counts[element] = 1

print(element_counts)

{'adm.cens.exit': 100, 'pneu': 97, 'Sex': 100, 'Age': 100, 'OS': 100, 'Status': 100, 'event_3': 44}


### C-index

In [137]:
print(round(np.mean(linear_results['train_C_index']),4))
print(round(np.std(linear_results['train_C_index']),4))
print(round(np.mean(linear_results['test_C_index']),4))
print(round(np.std(linear_results['test_C_index']),4))

0.5664
0.0068
0.5657
0.0137


In [131]:
print(round(np.mean(clinical_results['train_C_index']),4))
print(round(np.std(clinical_results['train_C_index']),4))
print(round(np.mean(clinical_results['test_C_index']),4))
print(round(np.std(clinical_results['test_C_index']),4))

0.6439
0.043
0.5847
0.0145


In [132]:
print(round(np.mean(ensemble_cox_results['train_C_index']),4))
print(round(np.std(ensemble_cox_results['train_C_index']),4))
print(round(np.mean(ensemble_cox_results['test_C_index']),4))
print(round(np.std(ensemble_cox_results['test_C_index']),4))

0.611
0.0173
0.5844
0.0151


In [133]:
print(round(np.mean(ensemble_aft_results['train_C_index']),4))
print(round(np.std(ensemble_aft_results['train_C_index']),4))
print(round(np.mean(ensemble_aft_results['test_C_index']),4))
print(round(np.std(ensemble_aft_results['test_C_index']),4))

0.612
0.0169
0.5844
0.0142
